In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import keras
import innvestigate
import importlib
from sklearn.model_selection import train_test_split
from keras import models, layers, regularizers, optimizers, losses

In [3]:
def build_model_keras(lr=0.0001, conv_filters=16, dense_neurons=32, dense_layers=1, activity_reg=0.0001, dropout_rate=0.2, input_channels=1):
    
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(121, 121, input_channels)))
    
    model.add(layers.Conv2D(conv_filters, (3, 3), activity_regularizer=regularizers.l2(0.01)))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(dropout_rate))
    
    model.add(layers.Conv2D(conv_filters, (3, 3), activity_regularizer=regularizers.l2(activity_reg)))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(dropout_rate))
    
    model.add(layers.Flatten())

    for _ in range(dense_layers):
        model.add(layers.Dense(dense_neurons, activity_regularizer=regularizers.l2(activity_reg)))
        model.add(layers.Activation('relu'))
    
    model.add(layers.Dense(2, activation='softmax'))
    
    model.compile(
        loss=losses.categorical_crossentropy,
        optimizer=optimizers.Adam(lr=lr),
        metrics=['categorical_accuracy']
    )
    
    return model

def onehot(x):
    y = np.zeros((x.size, x.max()+1)) 
    y[np.arange(x.size),x] = 1
    
    return(y)

In [4]:
model_keras = build_model_keras()
model_keras.load_weights('../processed_data/trained_weight.h5')

In [5]:
data_x=xr.open_dataset('../input_data/Vertical_Integrated_Moisture_convergence.nc')
data_y=pd.read_csv('../processed_data/extreme_event_days.csv')
data_y['time'] = pd.to_datetime(data_y['time'], errors='coerce')
data_y=data_y.drop('Unnamed: 0', axis=1)
n_time=len(data_x.time.values)
n_lat=len(data_x.lat.values)
n_lon=len(data_x.lon.values)
x_data=data_x.VIMFC.values
x_data=(x_data-x_data.mean(axis=0))/x_data.std(axis=0)
x_data=x_data.reshape(n_time, n_lat, n_lon, 1).astype(np.float32)
y_data=np.array(data_y['label'])
y_data=y_data.astype(np.int32)
y_data_one_hot=onehot(y_data)
ind = np.arange(len(y_data))
x_train, x_test, y_train, y_test, ind_train, ind_test = train_test_split(x_data, y_data_one_hot, ind, test_size=0.20, random_state=42, 
                                                                         shuffle = True, stratify = y_data_one_hot)
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(11688, 121, 121, 1) (2922, 121, 121, 1)
(11688, 2) (2922, 2)


In [6]:
model_keras.evaluate(x_test, y_test)

2922/2922 [==============================] - 4s 1ms/step


[1126.4574721569397, 0.9548254620123203]

In [7]:
predict_df = pd.read_csv('../processed_data/model_predictions.csv')
test_predictions = predict_df.loc[predict_df.set == 'test']
test_predictions_keras = np.argmax(model_keras.predict(x_data)[np.sort(ind_test)], axis = 1)
np.sum(test_predictions_keras != test_predictions.predicted_class)

In [22]:
model_strip = innvestigate.utils.model_wo_softmax(model_keras)

In [9]:
predict_df_pos=predict_df[predict_df['predicted_class']==1]
predict_df_pos_sort=predict_df_pos.sort_values(by='prob_1', ascending=False)
time=np.array(pd.to_datetime(predict_df_pos_sort['date']))
data_expos=data_x.where(data_x.time.isin([time]), drop=True)
data_ex=data_expos.VIMFC.values
data_ex=data_ex.reshape(1382, 121, 121, 1).astype(np.float32)

In [44]:
lrp_analyzerA1B0 = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPAlpha1Beta0(model_strip)
relevance = lrp_analyzerA1B0.analyze(data_ex)
relevance_data = xr.Dataset(
    data_vars=dict(
        vimfc=(['time', 'lat', 'lon'], relevance[:,:,:,0]), 
    ),
    coords=dict(
        time=data_expos.time.values, 
        lat = data_expos.lat.values, 
        lon = data_expos.lon.values),
)
relevance_data.to_netcdf('../processed_data/lrp_relevance_pos_days.nc')

In [24]:
predict_df_neg=predict_df[predict_df['predicted_class']==0]
predict_df_neg_sort=predict_df_neg.sort_values(by='prob_0', ascending=False)
time=np.array(pd.to_datetime(predict_df_neg_sort['date']))
data_exneg=data_x.where(data_x.time.isin([time]), drop=True)
data_ex=data_exneg.VIMFC.values
data_ex=data_ex.reshape(13228, 121, 121, 1).astype(np.float32)

# data_exneg=data_x.where(data_x.time.dt.year==1998, drop=True)
# data_exneg=data_exneg.where(data_exneg.time.dt.month==4, drop=True)
# data_exneg=data_exneg.where(data_exneg.time.dt.day==21, drop=True)
# data_exnegg=data_exneg.VIMFC.values
# data_exnegg=data_exnegg.reshape(1, 121, 121, 1).astype(np.float32)

In [25]:
lrp_analyzerA1B0 = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPAlpha1Beta0(model_strip)
relevance = lrp_analyzerA1B0.analyze(data_exnegg)
relevance_data = xr.Dataset(
    data_vars=dict(
        vimfc=(['time', 'lat', 'lon'], relevance[:,:,:,0]), 
    ),
    coords=dict(
        time=data_exneg.time.values, 
        lat = data_exneg.lat.values, 
        lon = data_exneg.lon.values),
)
relevance_data.to_netcdf('../processed_data/lrp_relevance_neg_days.nc')